<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Turku_ERA5_CDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install "cdsapi>=0.7.7"

In [2]:
!pip install netcdf4

In [3]:
import os

# Updated configuration: removed the UID prefix as per the error instructions
content = """url: https://cds.climate.copernicus.eu/api
key: c025f203-5930-4d9c-acd6-699c46be7fd8"""

with open(os.path.expanduser('~/.cdsapirc'), 'w') as f:
    f.write(content)

print("Configuration updated! Now attempting to update the library...")

# Also update your library to the latest version to match the new API
!pip install --upgrade cdsapi

Configuration updated! Now attempting to update the library...


In [ ]:
import cdsapi
import os
import calendar

c = cdsapi.Client()

# Jan 2024 to Jan 2025
tasks = [('2024', str(m).zfill(2)) for m in range(1, 13)] + [('2025', '01')]

for i, (year, month) in enumerate(tasks):
    filename = f'turku_{year}_{month}.nc'

    # Progress check (Plain Text)
    print(f"--- Task {i+1}/13: Processing {month}/{year} ---")

    if os.path.exists(filename):
        print(f"Skipping {filename}, already exists.")
        continue

    last_day = calendar.monthrange(int(year), int(month))[1]
    days = ['01'] if (year == '2025' and month == '01') else [str(d).zfill(2) for d in range(1, last_day + 1)]

    try:
        c.retrieve(
            'reanalysis-era5-land',
            {
                'variable': [
                    '2m_temperature',
                    '10m_u_component_of_wind',
                    '10m_v_component_of_wind',
                    'surface_solar_radiation_downwards',
                ],
                'year': year,
                'month': month,
                'day': days,
                'time': [f"{str(h).zfill(2)}:00" for h in range(24)],
                'area': [60.5, 22.5, 60.5, 22.5],
                'format': 'netcdf',
            },
            filename)
        print(f"Successfully saved {filename}")
    except Exception as e:
        print(f"Error on {month}/{year}: {e}")

print("All downloads finished!")

2025-12-19 17:54:32,264 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
INFO:ecmwf.datastores.legacy_client:[2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.


--- Task 1/13: Processing 01/2024 ---


2025-12-19 17:54:32,601 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2025-12-19 17:54:32,603

In [6]:
!head -n 5 /content/turku_2024_01.nc

PK    Ȍ�[�
��.� � 	   data_0.nc  ���HDF

�"            �_      >`      d`      
 �d      		      5                @       �          @                                                   +                                               


In [5]:
import xarray as xr
import pandas as pd
import numpy as np

def process_turku_data(file_list):
    all_data = []

    for file in file_list:
        ds = xr.open_dataset(file)
        df = ds.to_dataframe().reset_index()
        all_data.append(df)

    # Combine all months
    full_df = pd.concat(all_data)

    # 1. Convert Temperature (Kelvin to Celsius)
    full_df['temp_c'] = full_df['t2m'] - 273.15

    # 2. Calculate Wind Speed (sqrt(u^2 + v^2))
    full_df['wind_speed'] = np.sqrt(full_df['u10']**2 + full_df['v10']**2)

    # 3. Convert GHI (Joules/m2 to Watts/m2)
    # ERA5-Land SSRD is accumulated hourly
    full_df['ghi_w_m2'] = full_df['ssrd'] / 3600

    # Clean up columns
    result = full_df[['valid_time', 'temp_c', 'wind_speed', 'ghi_w_m2']]
    return result

# Example usage (after files download):
# files = ['turku_2024_01.nc', 'turku_2024_02.nc']
# df_final = process_turku_data(files)
# df_final.to_csv('turku_weather_final.csv', index=False)